## RNN
This model is a simple 3 layer RNN followed by a fully connected layer. We found the GRUs worked better than LSTMs, besides resulting in faster train times. We arrived at these parameters through [hyperparameter-tuning](../hyperparameter-tuning.ipynb).

![Network](bilstm_ptbdb.png)

In [1]:
import pandas as pd
import numpy as np

import tensorflow
from tensorflow.keras import optimizers, losses, activations, models
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, LearningRateScheduler, ReduceLROnPlateau
from tensorflow.keras.layers import Dense, Input, Dropout, GRU, \
    concatenate, Add, Activation
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import plot_model
from sklearn.utils.class_weight import compute_class_weight

from sklearn.metrics import f1_score, accuracy_score, roc_auc_score, precision_recall_curve, auc, confusion_matrix
from sklearn.model_selection import train_test_split
from IPython.display import Image

### Import data

In [2]:
df_1 = pd.read_csv("../data/ptbdb_normal.csv", header=None)
df_2 = pd.read_csv("../data/ptbdb_abnormal.csv", header=None)
df = pd.concat([df_1, df_2])

df_train, df_test = train_test_split(df, test_size=0.2, random_state=1337, stratify=df[187])

Y = np.array(df_train[187].values).astype(np.int8)
X = np.array(df_train[list(range(187))].values)[..., np.newaxis]

Y_test = np.array(df_test[187].values).astype(np.int8)
X_test = np.array(df_test[list(range(187))].values)[..., np.newaxis]

### Load model and skip to [evaluation](#evaluate)

In [2]:
train = False
model = load_model("bilstm_ptbdb_model")

### Or ALTERNATIVELY train it

In [8]:
seq_len = 187

def get_model(n_class=1, dropout=0.2, rnn_sizes = [128, 128], fc_sizes=[64], batch_norm=True):
    model = Sequential()
    model.add(Input(shape=(187, 1)))
    
    if batch_norm:
        model.add(BatchNormalization())
        
    for index, dim in enumerate(rnn_sizes):
        model.add(GRU(dim, dropout=dropout, return_sequences=(index != len(rnn_sizes) - 1)))
        
        if batch_norm:
            model.add(BatchNormalization())
    
    for index, dim in enumerate(fc_sizes):
        model.add(Dense(dim, activation="relu"))
        model.add(Dropout(dropout))
        
        if batch_norm:
            model.add(BatchNormalization())
            
    model.add(Dense(n_class, activation="sigmoid"))

    return model

In [ ]:
if train:
    model = get_model(n_class=1, 
                      dropout=0.2, 
                      rnn_sizes=[256, 256], 
                      fc_sizes=[64], 
                      batch_norm=True)
    file_path = "bilstm_ptbdb_retrain.h5"
    checkpoint = ModelCheckpoint(file_path, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
    early = EarlyStopping(monitor="val_accuracy", mode="max", patience=5, verbose=1)
    redonplat = ReduceLROnPlateau(monitor="val_accuracy", mode="max", patience=3, verbose=2)
    callbacks_list = [checkpoint, early, redonplat]  # early

    model.fit(X, Y, epochs=1000, verbose=1, callbacks=callbacks_list, validation_split=0.1)
    model.load_weights(file_path)

### Evaluate
<a id='evaluate'></a>

In [6]:
pred_test = model.predict(X_test)
pred_test = (pred_test>0.5).astype(np.int8)

f1 = f1_score(Y_test, pred_test)

print("Test f1 score : %s "% f1)

acc = accuracy_score(Y_test, pred_test)

print("Test accuracy score : %s "% acc)

auc_roc = roc_auc_score(Y_test, pred_test)

print("AUROC score : %s "% auc_roc)

precision, recall, _ = precision_recall_curve(Y_test, pred_test)

auc_prc = auc(recall, precision)
print("AUPRC score : %s "% auc_prc)

print(confusion_matrix(Y_test, pred_test))

Test f1 score : 0.9818524628800376 
Test accuracy score : 0.9735486087255238 
AUROC score : 0.959633770415838 
AUPRC score : 0.9851989057098708 
[[ 751   58]
 [  19 2083]]
